In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pylab
import scipy
import plotly
from plotly import graph_objs as go
from plotly.offline import iplot, plot, init_notebook_mode

In [3]:
def parseData(file):
    import csv
    Data =[]
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for lines in csv_reader:
           Data.append(lines)
        mainRootPoints =[]
        c=1
        while len(Data[c]) != 1:
            point = [float(Data[c][0]),float(Data[c][1])]
            mainRootPoints.append(point)
            c= c+1
        pqs =[]
        pq=[]
        c=c+1
        while c < len(Data):
            if len(Data[c]) ==1:
                pqs.append(pq)
                pq =[]
            if len(Data[c]) ==2:
                point = [float(Data[c][0]),float(Data[c][1])]
                pq.append(point)
            c = c+1
        pqs.append(pq)
    return (mainRootPoints, pqs)

In [32]:
def readArborFile(fileName):
    Array = parseData(fileName)
    mainRootPoints = Array[0]
    lateralRoots =Array[1]
    Plant = nx.Graph()
    #adding the mainroot base to graph
    Plant.add_node(0)
    Plant._node[0]['label']= 'main root base'
    Plant._node[0]['coordinate'] = [mainRootPoints[0][0], mainRootPoints[0][1]]
    Plant._node[0]['root number']= 0
    #constructing the main root
    for i in range(1, len(mainRootPoints)):
        Plant.add_node(i)
        Plant._node[i]['label']= 'main root'
        Plant._node[i]['coordinate'] = (mainRootPoints[i][0], mainRootPoints[i][1])
        Plant._node[i]['root number']=0
        Plant.add_edge(i-1,i)
        Plant[i-1][i]['label'] = 'main root'
        Plant[i-1][i]['G'] = 0
        Plant[i-1][i]['root number']=0
    #this way we can have individual labels for points
    count = len(mainRootPoints)
    rootNumber =1
    #constructing the lateral roots
    for root in lateralRoots:
        # constructing a lateral root
        #making the first point
        Plant.add_node(count)
        Plant._node[count]['label'] = 'lateral root'
        Plant._node[count]['coordinate'] = (root[0][0],root[0][1])
        Plant._node[count]['root number'] = rootNumber
        #making the inside points and edges
        for i in range(1,len(root) -1):
            c =count +i
            Plant.add_node(c)
            Plant._node[c]['label'] = 'lateral root'
            Plant._node[c]['coordinate'] = (root[i][0],root[i][1])
            Plant._node[c]['root number'] = rootNumber
            Plant.add_edge(c-1,c)
            Plant[c-1][c]['label'] = 'lateral root'
            Plant[c-1][c]['G']=0
            Plant[c-1][c]['root number'] = rootNumber
        #making the root tip
        count = count +len(root)-1
        Plant.add_node(count)
        Plant._node[count]['label'] = 'lateral root tip'
        Plant._node[count]['coordinate'] = (root[-1][0],root[-1][1])
        Plant._node[count]['root number'] = rootNumber
        Plant.add_edge(count-1,count)
        Plant[count-1][count]['label'] = 'lateral root'
        Plant[count-1][count]['G'] =0
        Plant[count-1][count]['root number'] = rootNumber
        count = count+1
        rootNumber = rootNumber+1
    # return a networkX graph of the arbor
    return Plant

In [33]:
def optimalArbor(arbor, G, alpha):
    MainRootPoints = []
    pqs=[]
    Plant = nx.Graph()
    c=0
    for node in arbor.nodes():
        x, y = arbor.nodes[node]['coordinate']
        if 'main root base' == arbor.nodes[node]['label']:
            MainRootPoints.append([x,y])
            Plant.add_node(c)
            Plant._node[c]['label'] = arbor.nodes[node]['label']
            Plant._node[c]['coordinate'] = arbor.nodes[node]['coordinate']
            Plant._node[c]['root number'] =arbor.nodes[node]['root number']
            c = c+1
        elif 'main root' == arbor.nodes[node]['label']:
            MainRootPoints.append([x,y])
            Plant.add_node(c)
            Plant._node[c]['label'] = arbor.nodes[node]['label']
            Plant._node[c]['coordinate'] = arbor.nodes[node]['coordinate']
            Plant.add_edge(c-1,c)
            Plant[c-1][c]['label'] = 'main root'
            Plant[c-1][c]['G'] =0
            Plant[c-1][c]['root number'] =0
            c = c+1
        elif 'lateral root tip' == arbor.nodes[node]['label']:
            pqs.append([x,y])
            Plant.add_node(c)
            Plant._node[c]['label'] = arbor.nodes[node]['label']
            Plant._node[c]['coordinate'] = arbor.nodes[node]['coordinate']
            c = c+1
    rootNumber =1
    for pq in pqs:
        Array =findMostOptimal(MainRootPoints,G, alpha,pq) 
        txy =Array[0]
        xs=Array[1]
        ys=Array[2]
        Plant.add_node(c)
        Plant._node[c]['label']='optimal'
        Plant._node[c]['root number'] = rootNumber
        tx = txy[0][0]
        ty = txy[1][0]
        Plant._node[c]['coordinate'] = [tx,ty]
        Plant._node[c]['pq'] = pq
        for node in Plant.nodes():
            if 'lateral root tip' == Plant.nodes[node]['label']:
                if (pq[0],pq[1]) == Plant.nodes[node]['coordinate']:
                    Plant.add_edge(c, node)
                    Plant[c][node]['label'] ='lateral root'
                    Plant[c][node]['root number'] = rootNumber
                    Plant[c][node]['G'] = G
                    Plant[c][node]['xs'] = xs
                    Plant[c][node]['ys']= ys
                    break
        rootNumber = rootNumber +1
        c=c+1
    return Plant
    # arbor is a networkx graph
    # return a networkx graph representing optimal arbor

In [34]:
def findMostOptimal(MainRootPoints,G, al,pq):
    p = pq[0]
    q = pq[1]
    point1 = MainRootPoints[0]
    point2 = MainRootPoints[1]
    CD = 0
    bestDAC = findOptimalInSegment(point1, point2,G,al,pq,CD) 
    #return length, txy, xs,ys
    bestLength = bestDAC[0] 
    for i in range(2, len(MainRootPoints)):
        length = findOptimalInSegment(point1, point2,G,al,pq,CD)[0]
        if length < bestLength:
            bestDAC = findOptimalInSegment(point1, point2,G,al,pq,CD)
            #returns length, txy, xs, ys
            bestLength = length
        CD = CD +findOptimalInSegment(point1, point2,G,al,pq,CD)[4] 
        point1 = point2
        point2 = MainRootPoints[i]
    return (bestDAC[1:])
    #returen [txy,xs,ys]

In [4]:
def findTheta (a,b,c,d):
    theta = np.arctan((np.absolute(a-c))/(np.absolute(b-d)))
    m = (b-d)/(a-c)
    if m > 0:
        theta =-theta
    return theta
def rotateLine(xs,ys,theta,a,b):
    nx=[]
    ny=[]
    R = [[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]]
    for i in range(0,len(xs)):
        A =[xs[i]-a,ys[i]-b]
        xy=np.array(np.matmul(A,R))
        nx.append(xy[0])
        ny.append(xy[1])
    return(nx, ny)
def findBest(Tmax,G,p,q,a):
    t=0
    Tmax = -Tmax
    bestLength= 100000000000000000000.0
    bestT = t
    while t >= Tmax:
        b = (q - G * (p**2) - t)/p
        if p>0:
            x =np.linspace(0, p, num=50)
        elif p < 0:
            x =np.linspace(p, 0, num=50)
        lot = np.sqrt(1+(2*G*x + ((q-G*(p**2)-t)/p))**2)
        y= a*(lot) +(1-a)*(lot - t)
        length = scipy.integrate.trapezoid(y,x)
        #print(length)
        if length < bestLength:
            #ploting(Tmax,p,q,bestT,G) #comparing possible lines
            bestLength = length
            bestT = t
        t = t - 0.01 #last 
    #ploting(Tmax,p,q,bestT,G)
    #print("length: " + str(bestLength) +" t= " + str(bestT))
    return(bestLength,bestT)
def reAngle(xs, ys, theta, a,b):
    nx =[]
    ny=[]
    R = [[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]]
    inR = np.linalg.inv(R)
    ab= [a,b]
    for i in range(0, len(xs)):
        Ap = [xs[i],ys[i]]
        xy=np.array(np.add(np.matmul(Ap,inR),ab))
        nx.append(xy[0])
        ny.append(xy[1])
    return(nx, ny)
def deAngleCurve(G,al,p,q,theta,a,b,c,d):
    pq = rotateLine([p],[q], theta, a,b)
    pn = pq[0][0]
    qn = pq[1][0]
    tmax = rotateLine([c],[d], theta, a,b)
    tmax = tmax[1][0]
    best = findBest(tmax, G,pn,qn,al) #returns bestLength and bestT
    dPlotA = plotDeAngle(tmax, G, pn ,qn,best[1])
    dPlot = go.Scatter(x = dPlotA[0], y = dPlotA[1], mode= 'lines' , name = str(best[0]))
    raPlotA = reAngle(dPlotA[0], dPlotA[1],theta,a,b)
    raPlot = go.Scatter(x = raPlotA[0], y = raPlotA[1], mode = 'lines', name = str(best[0]))
    txy = reAngle([0],[best[1]], theta,a,b)
    return(best[0], txy, dPlot,raPlot)
def plotDeAngle (tmax,G,p,q,t):
    x = np.linspace(0,p)
    if p <0:
        x = np.linspace(p,0)
    b=(q - G*(p**2) -t)/p
    y=G * x**2 + b*x + t
    return (x,y)

In [5]:
def findOptimalInSegment(point1, point2,G,al,pq):
    a =point1[0]
    b= point1[1]
    c= point2[0]
    d = point2[1]
    p=pq[0]
    q=pq[1]
    theta = findTheta (a,b,c,d)
    ArrayDAC = deAngleCurve(G,al,p,q,theta,a,b,c,d) #returns length, txy, dplot, raplot
    return ArrayDAC

In [22]:
def findMostOptimal(main, G,al,pq):
    p = pq[0]
    q = pq[1]
    point1 = main[0]
    point2 = main[1]
    bestDAC = findOptimalInSegment(point1, point2,G,al,pq)#returns length, txy, dplot, raplot
    bestLength = bestDAC[0]
    #print(bestLength)
    point1 = point2
    for i in range(2, len(main)):
        point2 = main[i]
        length = findOptimalInSegment(point1, point2,G,al,pq)[0]
        if length < bestLength:
            bestDAC = findOptimalInSegment(point1, point2,G,al,pq)
        point1 = point2
    return (bestDAC)

In [7]:
def plotPoints(points, name):
    xs = []
    ys=[]
    for i in points:
        xs.append(i[0])
        ys.append(i[1])
    trace1 = go.Scatter(x=xs, y = ys, name = name)
    return (trace1)

In [8]:
def plotData(array):
    mainRootPoints = array[0]
    pqs = array[1]
    #print(pqs)
    fig = go.Figure()
    trace= plotPoints(mainRootPoints, "main root")
    fig.add_trace(trace)
    for pq in pqs:
        trace = plotPoints(pq, "lateral root")
        fig.add_trace(trace)
    return (fig)

In [27]:
array = parseData('084_4_C_day1.csv')
fig = plotData(array)
fig.show()
fig2 = go.Figure()
G = 2
al = 1
lateralRoots = array[1]
MainRootPoints = array[0]
mainRootBase = MainRootPoints[0]
trace= plotPoints(MainRootPoints, "main root")
fig2.add_trace(trace)
pqs =[]
Info = [["pq", "length", "txy"]]
for root in latteralRoots:
    pqs.append(root[-1])
for pq in pqs:
    DAC = findMostOptimal(MainRootPoints, G,al,pq) #returns length, txy, dplot, raplot
    array =[pq, DAC[0], DAC[1]]
    Info.append(array)
    fig.add_trace(DAC[3])
    fig2.add_trace(DAC[3])
fig.show()
fig2.show()
print(Info)

[['pq', 'length', 'txy'], [[6.763005, 4.337383], 0.31060960169400276, ([7.053897704400315], [4.292200457561887])], [[7.439422, 4.27106], 0.46673136243388197, ([7.107893912576261], [4.589498031540513])], [[7.287631, 4.43411], 0.4355615947021112, ([7.097158662956166], [4.825790746971235])]]


In [28]:
csv = '084_4_C_day1.csv'

def pythag(point1,point2):
    x1, y1 = point1
    x2, y2 = point2
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

def lengthMR(mainRoot):
    num_points = len(mainRoot)
    distances = [[0] * num_points for _ in range(num_points)]
    mainRootTotal = 0
    for i in range(num_points):
        for j in range(i + 1, num_points):
            distance = pythag(mainRoot[i], mainRoot[j])
            mainRootTotal += distance
        print (mainRootTotal)
        return mainRootTotal

def sumLatRoots(Info):
    num = len(Info)
    latRootTotal = 0
    for i in range(1, num):
        length = Info[i][1]
        latRootTotal += length
    print (latRootTotal)
    return latRootTotal

def sumWiringCostOpt(csv):
    array = parseData(csv)
    mainRoot = array[0]
    wiringCost = lengthMR(mainRoot) + sumLatRoots(Info)
    return wiringCost

In [29]:
#csv = input("Enter the CSV file name: ")
sumWiringCostOpt(csv)

17.666441408287636
1.212902558829996


18.87934396711763

In [31]:
array = parseData('084_4_C_day1.csv') #using cell[2]
#returns [mainrootpoints, lateralRoot points]
fig = plotData(array) #cell[3]
fig.show()
fig2 = go.Figure()
fig3 =go.Figure()
G= 2
al =.7
latteralRoots = array[1]
MainRootPoints = array[0]
trace= plotPoints(MainRootPoints, "main root") #cell[4]
fig2.add_trace(trace)
pqs =[]
Info = [["pq", "length", "txy"]]
for root in latteralRoots:
    pqs.append(root[-1]) #takes the root tip point of each lateral root segment
for pq in pqs:
    DAC = findMostOptimal(MainRootPoints, G,al,pq) #cell[5]
    #returns length, txy, dplot, raplot
    array =[pq, DAC[0], DAC[1]]
    Info.append(array)
    fig.add_trace(DAC[3])
    fig2.add_trace(DAC[3])
    fig3.add_trace(DAC[2])
fig.show()
fig2.show()
fig3.show()
#print(Info)
print(latteralRoots)

[[[7.03248, 4.136673], [6.921742, 4.219422], [6.763005, 4.337383]], [[7.065898, 4.198863], [7.312281, 4.231294], [7.439422, 4.27106]], [[7.125593, 4.363359], [7.182462, 4.368823], [7.287631, 4.43411]]]
